# todo: replace nan's with 0
# todo: and do the calculation again

and add the above to the caveat section. Here is a caveat based on missing/bad data

In [1]:
import pandas as pd
import os
import numpy as np
import cpi

/var/folders/ql/2vqj8p0d1gq1gggdp2dycp5c0000gn/T/ipykernel_95530/1910333970.py:4: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  import cpi


In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
os.getcwd()

'/Users/hashim/Library/Mobile Documents/com~apple~CloudDocs/ust/seis732-data stores and feature design/project'

In [4]:
df_employment = pd.read_csv('city_employment/mpls_zipcode_employment_summary.csv')
df_flour = pd.read_excel('raw data/flour_midwest_avg_price_not_seasonally_adjusted.xlsx', skiprows=9)
df_rice = pd.read_excel('raw data/rice_midwest_avg_price_not_seasonally_adjusted.xlsx', skiprows=9)

/opt/anaconda3/envs/py39/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/envs/py39/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# NOTE: 

There is no data for 2001 for rice data. I am going to take the average of 2000 and 2002 and just add it in there

In [5]:
df_rice.loc[10:11].mean()['Jan':'Dec'].mean()

0.4764166666666667

In [6]:
# New row data for the year 2001 with all months having a value of 0.476
new_row = {'Year': 2001, 'Jan': 0.476, 'Feb': 0.476, 'Mar': 0.476, 'Apr': 0.476, 'May': 0.476, 
           'Jun': 0.476, 'Jul': 0.476, 'Aug': 0.476, 'Sep': 0.476, 'Oct': 0.476, 'Nov': 0.476, 'Dec': 0.476}

In [7]:
df_rice = pd.concat([df_rice, pd.DataFrame([new_row])], ignore_index=True)

In [8]:
df_rice.sort_values('Year', inplace=True)
df_rice.reset_index(drop=True, inplace=True)

In [9]:
years = pd.Series(list(range(1990,2024)))

In [10]:
flour_avg_price = df_flour.T.loc['Jan':'Dec',].mean()
rice_avg_price = df_rice.T.loc['Jan':'Dec', ].mean()

In [11]:
years.name = 'Year'
flour_avg_price.name = 'Flour Avg Price per lb'
rice_avg_price.name = 'Rice Avg Price per lb'

In [12]:
df_avg_price = pd.concat([years, flour_avg_price, rice_avg_price],axis=1)

In [13]:
# #f_avg_price.apply(lambda x: cpi.inflate(x['Flour Avg Price per lb'], x['Year']), axis=1)
# df_avg_price['Adjusted Flour Avg Price per lb'] = df_avg_price.apply(lambda x: cpi.inflate(x['Flour Avg Price per lb'], x['Year']), axis=1)
# df_avg_price['Adjusted Rice Avg Price per lb'] = df_avg_price.apply(lambda x: cpi.inflate(x['Rice Avg Price per lb'], x['Year']), axis=1)

In [14]:
df_merged = pd.merge(df_employment, df_avg_price, on=['Year'], how='inner')

## Feature Engineering

Going to calculate number of food service jobs in each zip code by year. 

In [15]:
df_merged['% Food Service Jobs in Zip'] = df_merged['Employment for NAICS 722']/df_merged['Total Employment in Zipcode']
df_merged['% Food Service Establishments in Zip'] = df_merged['Establishments for NAICS 722']/df_merged['Total Establishments in Zipcode']

In [16]:
df_merged = df_merged.sort_values(by=['Zipcode', 'Year']).reset_index(drop=True)

Also going to calculate the percentage yearly change for the below metrics

In [17]:
columns_to_calculate = [
    'Total Employment in Zipcode', 
    'Total Establishments in Zipcode', 
    'Total Payroll in Zipcode', 
    'Employment for NAICS 722', 
    'Establishments for NAICS 722', 
    'Avg. Annual Wage for NAICS 722', 
    'Total Payroll for NAICS 722'
]

In [18]:
df_merged.fillna(0,inplace=True)

In [19]:
for column in columns_to_calculate:
    df_merged[column + ' Yearly % Change'] = df_merged.groupby('Zipcode')[column].pct_change() * 1

In [20]:
df_merged['Zipcode'].unique().tolist()

['55401-04;15,54,87,88',
 '55405,08,16',
 '55406',
 '55407',
 '55409,10,19',
 '55411',
 '55412,30',
 '55413,18,21',
 '55414,55',
 '55417']

## Map Zipcode to neighborhood to join with `geojson` data

Just Google'd the zip codes and found the relevant neighborhoods

In [21]:
map_zipcode_to_neighborhood = {
    '55401-04;15,54,87,88':'North Loop',
    '55405,08,16': 'Lowry Hill',
    '55406': 'Howe',
    '55407': 'Powderhorn Park',
    '55409,10,19': 'King Field',
    '55411': 'Near - North',
    '55412,30': 'Webber - Camden',
    '55413,18,21': 'Beltrami',
    '55414,55': 'University of Minnesota',
    '55417': 'Wenonah'
    
}

In [22]:
df_merged['Neighborhood'] = df_merged['Zipcode'].map(map_zipcode_to_neighborhood)

In [23]:
df_merged.fillna(0,inplace=True)
df_merged = df_merged.replace(np.inf, 1)

In [24]:
#df_merged.to_csv('employment_and_grocery_merged_updated.csv', index=False)

In [25]:
df_merged['Adjusted Avg. Annual Wage for NAICS 722'] = df_merged.apply(lambda x: cpi.inflate(x['Avg. Annual Wage for NAICS 722'], x['Year']), axis=1)
df_merged['Adjusted Flour Avg Price per lb'] = df_merged.apply(lambda x: cpi.inflate(x['Flour Avg Price per lb'], x['Year']), axis=1)
df_merged['Adjusted Rice Avg Price per lb'] = df_merged.apply(lambda x: cpi.inflate(x['Rice Avg Price per lb'], x['Year']), axis=1)

In [26]:
df_merged['Adjusted Avg. Annual Wage for NAICS 722 Yearly % Change'] = df_merged.groupby('Zipcode')['Adjusted Avg. Annual Wage for NAICS 722'].pct_change() * 1
df_merged['Adjusted Avg. Flour Avg Price per lb Yearly % Change'] = df_merged.groupby('Zipcode')['Adjusted Flour Avg Price per lb'].pct_change() * 1
df_merged['Adjusted Avg. Rice Avg Price per lb Yearly % Change'] = df_merged.groupby('Zipcode')['Adjusted Rice Avg Price per lb'].pct_change() * 1

In [27]:
df_merged['% Difference Avg. Wage NAICS 722 - Flour'] = df_merged['Adjusted Avg. Annual Wage for NAICS 722 Yearly % Change'] - df_merged['Adjusted Avg. Flour Avg Price per lb Yearly % Change']
df_merged['% Difference Avg. Wage NAICS 722 - Rice'] = df_merged['Adjusted Avg. Annual Wage for NAICS 722 Yearly % Change'] - df_merged['Adjusted Avg. Rice Avg Price per lb Yearly % Change']

In [28]:
#df_merged.to_csv('employment_and_grocery_merged_updated.csv', index=False)